## Capstone Notebook 

## IBM Data Science Capstone Project on Coursera¶

### San Diego reaserch on ZIP codes

### I. Description of the Project and Business Problem:
For this project we are assumming we are a company and we are trying to reach customers in San Diego. San Diego has a population estimated at 1.43 million. At the last census in 2010, San Diego's population was 1.307 million. San Diego has a population density of 4,003 people per square mile in the city proper.Starting out, it is difficult to know exactly who your target market is in San Diego, California. Even over time your ideal customers are subject to change. Therefore, it is important to evaluate your company and its products often to ensure your marketing campaigns are reaching the right people.

Business Problem is to analyse and select the best location in the city for our marketing campaigns targeted advertising is a way of placing ads based on demographics, on the consumers' previous buying history or on behavior. Many types of targeted advertising are used online, but advertisers use in other media as well. Examples of targeted advertising include having consumers choose which ads to view and placing ads on social networking sites and even billboards that vary depending on who is viewing them.

### Import/Install Required Libraries
 We will import our necessary packages and start pulling our data for data prep and usage

In [2]:
import pandas as pd
pd.set_option('max_rows', 300)
import numpy as np
! pip install geocoder
import geocoder as gc
import requests
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0
import folium
import random
import matplotlib.cm as cm
import matplotlib.colors as colors
print('--Imports Completed--')

     |████████████████████████████████| 102kB 12.1MB/s ta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                   

### San Diego ZIP codes 

### Retrieve/Format Table of CA Zip Codes

In [5]:
 #Download & Format Available California ( San Diego ) Zipcodes
url = 'https://www.geonames.org/postal-codes/US/CA/california.html'
sdData = pd.read_html(url)
sdData = sdData[2]  #Select Correct Table from url
sdData
sdData = sdData.drop(sdData.columns[0],axis=1) 
sdData = sdData.drop(sdData.columns[5:len(sdData.columns)],axis=1).reset_index(drop=True) #Drop extraneous columns
#Rename Columns
cols = list(sdData.columns)
cols[3] = 'State'
cols[4] = 'County'
sdData.columns = cols
sdData = sdData.drop(len(sdData.index)-1).reset_index(drop=True) # drop last row 

In [6]:
#extract coordinates fromm odd rows 
#reinsert as columns
lat = [] 
long = [] 
for count, n in enumerate(sdData['Place']):
    if (count % 2 != 0):
        temp = n.split('/')
        lat.append(temp[0])
        long.append(temp[1])
        sdData = sdData.drop(count)
sdData = sdData.reset_index(drop=True)

#add Latitude and Logitude to San Diego Dataframe
data = {'Latitude':lat,'Longitude':long}
temp = pd.DataFrame(data,dtype='float')
sdData = sdData.join(temp)
sdData = sdData[sdData['County'].str.contains('San Diego')] ## Select only zip codes in San Diego 


####  Remove Zipcodes with Duplicate Coordinates

In [7]:
sdData = sdData.sort_values(by = ['Latitude','Longitude']).reset_index(drop=True)

for (row) in range(len(sdData.index)-1):
    if sdData.loc[row,'Latitude'] == sdData.loc[row+1,'Latitude']:
        if sdData.loc[row,'Longitude'] == sdData.loc[row+1,'Longitude']:
            sdData = sdData.drop(row)

sdData = sdData.sort_values('Code').reset_index(drop=True)

#### Comple San Diego Zip codes

In [10]:
#Display Corrected Dataframe & its shape
print('Rows:',sdData.shape[0],'\nColumns:',sdData.shape[1])
sdData

Rows: 25 
Columns: 7


,Place,Code,Country,State,County,Latitude,Longitude
0,Chula Vista,91910,United States,California,San Diego,32.637,-117.068
1,Chula Vista,91911,United States,California,San Diego,32.608,-117.056
2,Chula Vista,91913,United States,California,San Diego,32.651,-116.985
3,Chula Vista,91914,United States,California,San Diego,32.659,-116.965
4,La Mesa,91941,United States,California,San Diego,32.760,-117.012
5,Spring Valley,91977,United States,California,San Diego,32.724,-116.998
6,Carlsbad,92008,United States,California,San Diego,33.160,-117.325
7,Carlsbad,92009,United States,California,San Diego,33.095,-117.262
8,El Cajon,92019,United States,California,San Diego,32.778,-116.919
9,El Cajon,92020,United States,California,San Diego,32.793,-116.967


### Map of Zip Code Locations 
#### This will show us all zip codes in San Diego 

In [12]:
# Get coordinates from map and calculating the Latitude and Longitude
address = 'San Diego, CA'
geolocator = Nominatim(user_agent="SanDiego_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(address,'\nLatitude:',latitude,'\nLongitude:',longitude)

San Diego, CA 
Latitude: 32.7174209 
Longitude: -117.1627714


### Creating a Map 

In [14]:
#Create Map
sdMap = folium.Map(location=[latitude, longitude],tiles = 'Stamen Terrain', zoom_start=10)

#add Zipcode labels to map
for lat, lng, code, place in zip(sdData['Latitude'], sdData['Longitude'], sdData['Code'], sdData['Place']):
    label = '{}: {}'.format(place, code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5,
        parse_html=False).add_to(sdMap)

#display map
display(sdMap)

#### Retrieve Venues via Foursquare API

In [15]:
#Define Foursquare Credentials
CLIENT_ID = 'Insert Your Foursquare ID' #Foursquare ID
CLIENT_SECRET = 'Insert Your Foursquare Secret' #Foursquare Secret
VERSION = '20180605'

In [16]:
#Define Foursquare Credentials
CLIENT_ID = 'MAACNYPG1YWNGXDEHPVM2U2KICHIZXKYPXFDDHA5HXYXQZDP' #Foursquare ID
CLIENT_SECRET = 'GMK12PCPN1HSMFOG103VGB4JQFKMCK0TL0PY2XL35GGX35KB' #Foursquare Secret
VERSION = '20180605'

In [17]:
CLIENT_ID = 'MAACNYPG1YWNGXDEHPVM2U2KICHIZXKYPXFDDHA5HXYXQZDP' # your Foursquare ID
CLIENT_SECRET = 'GMK12PCPN1HSMFOG103VGB4JQFKMCK0TL0PY2XL35GGX35KB' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MAACNYPG1YWNGXDEHPVM2U2KICHIZXKYPXFDDHA5HXYXQZDP
CLIENT_SECRET:GMK12PCPN1HSMFOG103VGB4JQFKMCK0TL0PY2XL35GGX35KB


In [18]:
## Caluculating from the city centre advertising companies

In [19]:
address = '326 Broadway, San Diego, CA 92101, United States'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

32.7159477 -117.16150502103


In [20]:
search_query = 'Advertising'
radius = 500
print(search_query + ' .... OK!')

Advertising .... OK!


In [21]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=MAACNYPG1YWNGXDEHPVM2U2KICHIZXKYPXFDDHA5HXYXQZDP&client_secret=GMK12PCPN1HSMFOG103VGB4JQFKMCK0TL0PY2XL35GGX35KB&ll=32.7159477,-117.16150502103&v=20180605&query=Advertising&radius=500&limit=30'

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dd15f707828ae0028fee56c'},
 'response': {'venues': [{'id': '4ec6921010818589608f07e3',
    'name': 'Farm Advertising',
    'location': {'address': '3300 3rd Ave',
     'lat': 32.71740841997721,
     'lng': -117.15656697750092,
     'labeledLatLngs': [{'label': 'display',
       'lat': 32.71740841997721,
       'lng': -117.15656697750092}],
     'distance': 490,
     'postalCode': '92103',
     'cc': 'US',
     'city': 'San Diego',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['3300 3rd Ave',
      'San Diego, CA 92103',
      'United States']},
    'categories': [],
    'referralId': 'v-1574002548',
    'hasPerk': False}]}}

In [23]:
#Function to Create Dataframe of Venues from Foursquare API
def getVenues(code, latitudes, longitudes, radius=500):
    
    venues_list=[]
    dropList = []  
    for code, lat, lng in zip(code, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            500)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        if results == []:
            print('Warning no venues returned for: ',code)
            dropList.append(code)
        # return only relevant information for each nearby venue
        venues_list.append([(
            code, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zipcode', 
                  'Zipcode Latitude', 
                  'Zipcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']  
    return(nearby_venues, dropList)

In [24]:
sdVenues, sdDrop = getVenues(sdData['Code'],sdData['Latitude'],sdData['Longitude'])

Warning no venues returned for:  92008


In [25]:
#Display Venues
sdVenues.head()

,Zipcode,Zipcode Latitude,Zipcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,91910,32.637,-117.068,SD Hardwood,32.637478,-117.067909,Home Service
1,91910,32.637,-117.068,JL Body Wrap Salon,32.636163,-117.066831,Massage Studio
2,91910,32.637,-117.068,Chula Vista Taco Catering,32.638544,-117.070559,Business Service
3,91911,32.608,-117.056,All Tech Services Elite,32.607694,-117.056494,Electronics Store
4,91911,32.608,-117.056,24minute Fitness,32.608634,-117.056155,Gym


### K-Cluster Analysis

In [26]:
#Create Dataframe for kClusters Analysis
sdEncode = pd.get_dummies(sdVenues[['Venue Category']], prefix="", prefix_sep="")
sdEncode['Zipcode'] = sdVenues['Zipcode']
fixed_columns = [sdEncode.columns[-1]] + list(sdEncode.columns[:-1])
sdEncode = sdEncode[fixed_columns]
sdSorted = sdEncode.groupby(by='Zipcode').mean().reset_index()
sdCluster = sdSorted.drop('Zipcode', axis=1)

### Average Venue Category by Zip Code

In [27]:
sdSorted.head(10)


,Zipcode,ATM,Accessories Store,American Restaurant,Arts & Crafts Store,Asian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,...,Thai Restaurant,Theater,Thrift / Vintage Store,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,91910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,91911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,91913,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,91914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,91941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,91977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,92009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,92019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,92020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,92025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Top 5 Venue Categories for each Zipcode

In [28]:
topVen = pd.DataFrame()
#Get top 5 venues for each Zipcode
for count, z in enumerate(sdSorted['Zipcode']):
    temp = sdSorted[sdSorted['Zipcode'] == z]  #Select all of a single zipcode
    temp = temp.drop('Zipcode',axis=1)
    temp.reset_index()
    temp = temp.sort_values(by=count,axis=1,ascending=False)
    topVen = topVen.append(pd.Series(temp.iloc[:,0:5].columns),ignore_index = True)
    
#Rename Columns & Display first 5 rows
topVen.columns = ['#1 Venue','#2 Venue','#3 Venue','#4 Venue','#5 Venue'] 
print(topVen.shape)
topVen.head()

(24, 5)


,#1 Venue,#2 Venue,#3 Venue,#4 Venue,#5 Venue
0,Massage Studio,Business Service,Home Service,ATM,Music Venue
1,Business Service,Electronics Store,Gym,Home Service,Health & Beauty Service
2,Home Service,ATM,Optical Shop,Movie Theater,Music Venue
3,Bank,Ice Cream Shop,Nail Salon,Home Service,Park
4,Plaza,Gym,Pub,Park,Coffee Shop


### Create k Clusters from Category Data

In [30]:
k = 8
kmeans = KMeans(k, random_state=0).fit(sdCluster)

In [31]:
#add Cluster Labels to Top Venues Dataframe
topVen.insert(0,'Cluster Labels',kmeans.labels_ + 1)

## Create Final Dataframe: Zipcode, Location Name, Coordinates, k Cluster Group, Top 5 Venues

In [32]:
# Join Zipcodes & Coordinates with Cluster Labels and Top Venues
sdFinal = pd.DataFrame()
sdFinal = sdData.loc[:,['Code','Place','Latitude','Longitude']].sort_values(by='Code').reset_index(drop=True)
for x in sdDrop:
    if sdFinal['Code'].str.contains(x).any():
        sdFinal = sdFinal.drop(sdFinal.index[sdFinal['Code']==x])
sdFinal = sdFinal.reset_index(drop=True)
sdFinal = sdFinal.join(topVen)
sdFinal = sdFinal.sort_values('Cluster Labels').reset_index(drop=True)
sdFinal

,Code,Place,Latitude,Longitude,Cluster Labels,#1 Venue,#2 Venue,#3 Venue,#4 Venue,#5 Venue
0,91914,Chula Vista,32.659,-116.965,1,Bank,Ice Cream Shop,Nail Salon,Home Service,Park
1,91913,Chula Vista,32.651,-116.985,2,Home Service,ATM,Optical Shop,Movie Theater,Music Venue
2,92026,Escondido,33.161,-117.098,3,Pizza Place,Convenience Store,Mexican Restaurant,ATM,Noodle House
3,92069,San Marcos,33.144,-117.170,3,Mexican Restaurant,Grocery Store,Gym,ATM,Moroccan Restaurant
4,92056,Oceanside,33.197,-117.283,3,Mobility Store,Convenience Store,Mexican Restaurant,Baseball Field,Optical Shop
5,91910,Chula Vista,32.637,-117.068,4,Massage Studio,Business Service,Home Service,ATM,Music Venue
6,91977,Spring Valley,32.724,-116.998,4,Business Service,Home Service,ATM,Optical Shop,Music Venue
7,92078,San Marcos,33.119,-117.185,5,Business Service,Cable Car,ATM,Optical Shop,Nail Salon
8,92054,Oceanside,33.207,-117.357,6,Convenience Store,Dog Run,Pet Store,ATM,Optical Shop
9,92109,San Diego,32.797,-117.241,7,Yoga Studio,Taco Place,Sushi Restaurant,Ice Cream Shop,Italian Restaurant


### Creating a MAP of clusters

In [36]:
##### initialize map
clusterMap = folium.Map(location=[latitude, longitude],tiles='Stamen Toner', zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**4 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
random.shuffle(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sdFinal['Latitude'], sdFinal['Longitude'], sdFinal['Code'], sdFinal['Cluster Labels']):
    label = folium.Popup(str(poi) + ': Cluster ' + str(cluster), parse_html=True)
    folium.Circle(
        [lat, lon],
        radius=500,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.75).add_to(clusterMap)
print('K    Count\n-----------')
print(sdFinal['Cluster Labels'].value_counts(sort=False))
display(clusterMap)

K    Count
-----------
1     1
2     1
3     3
4     2
5     1
6     1
7    14
8     1
Name: Cluster Labels, dtype: int64
